### A simple REPL

This is an example for a simple REPL:
1) Run a command (add text to a file)
2) Receive current diagnostics (whole file), goal and term goal

In [1]:
# Setup a new lean project in the following folder:
# Using the instructions from mathlib_warning_analysis.ipynb
LEAN_FOLDER = "LeanProject"

In [ ]:
# Install leanclient
!pip install --upgrade leanclient

In [2]:
import leanclient as lc


class REPL:
    """Simple file-based REPL for lean.

    For serious tasks extend this or consider using proper REPLs like:
    - LeanDojo
    - Pantograph
    """

    def __init__(self, lean_folder):
        # Write a temporary file as the basis for the REPL
        self.temp_file = lean_folder + "/temp_repl.lean"
        with open(lean_folder + "/" + self.temp_file, "w") as f:
            f.write("\n")

        self.lsp_client = lc.LeanLSPClient(lean_folder)
        self.client = self.lsp_client.create_file_client(self.temp_file)
        self.client.open_file()

        self.current_coords = (0, 0)

    def run_command(self, command: str) -> list:
        """Run a command in the REPL (add to the file).

        Args:
            command (str): The command to run.

        Returns:
            list: A list of diagnostics, goal, and term_goal after running the command.
        """
        self.client.update_file(
            [
                lc.DocumentContentChange(
                    command, self.current_coords, self.current_coords
                )
            ]
        )
        diagnostics = self.client.get_diagnostics()

        # Update the current coordinates
        split_command = command.split("\n")
        num_lines = len(split_command)
        num_chars = (
            len(split_command[-1])
            if num_lines > 1
            else len(command) + self.current_coords[1]
        )
        self.current_coords = (self.current_coords[0] + num_lines - 1, num_chars)

        goal = self.client.get_goal(*self.current_coords)
        term_goal = self.client.get_term_goal(*self.current_coords)

        goal = goal["goals"] if goal is not None else None
        term_goal = term_goal["goal"] if term_goal is not None else None

        return [diagnostics, goal, term_goal]

In [3]:
import random
import time

# We test the REPL by running sections of a valid mathlib file
path = f"{LEAN_FOLDER}/.lake/packages/mathlib/Mathlib/FieldTheory/AbelRuffini.lean"

with open(path, "r") as f:
    content = f.read()

commands = []
while content:
    num_chars = random.randint(10, 1500)
    command, content = content[:num_chars], content[num_chars:]
    commands.append(command)

print(f"Split {path} into {len(commands)} commands.")

Split LeanProject/.lake/packages/mathlib/Mathlib/FieldTheory/AbelRuffini.lean into 23 commands.


In [4]:
# Run all commands
repl = REPL(LEAN_FOLDER)

t0 = time.time()
intermediate_goals = []
for i, command in enumerate(commands):
    diag, goal, term_goal = repl.run_command(command)
    intermediate_goals.append(goal)

print(
    f"Ran {len(commands)} commands: {len(commands) / (time.time() - t0):.2f} commands per second."
)
print("Final diagnostics after running all commands:", diag)
print("Intermediate goals:")
for i, goal in enumerate(intermediate_goals):
    print(f"Command {i + 1}: {goal}")

RuntimeError: This event loop is already running